In [ ]:
#Imports
from __future__ import division

import random
import numpy as np
import plotly.graph_objects as go

from IPython.display import Markdown as md

In [ ]:
##aquire the data with the following global variables

N = 100 #people participate in the market
start_cash = 100.00	#the starting budget per person
rounds = 500		#investment round that the particpants will partake in

if (N%2):
    print "please keep N even for smooth operations"

def func_bound_exp_p(mean):
    chance = 2
    while chance > 1:
        chance = np.random.exponential(mean)
    return chance

def func_redistr_wealth(people, d):
    budget = 0
    for i in range(N):
        take = people[i][-1] * d
        people[i][-1] -= take
        budget += take
    
    give = budget/N    
    for i in range(N):
        people[i][-1] += give
    
    return people
        

def func_inv_round(people, distr=0): 
    '''
    a single round of investments applied on the people list, returns the modified version
    distr: how much the each persons cash is taken and redistribued. 0 = nothing, 1 is everything.
    '''
    
    sN = range(N) #range of indexes of the people participating
    random.shuffle(sN) #shuffled indexes so a random 50% win (first half of the shuffled indexes), the other 50% loses

    # percentage market gain/loss this round, up to 50% win or loss 
    p = random.random() / 2 #for uniform volatility with a max of 50%
    
    for cnt, Pidx in enumerate(sN):
        if  cnt == int(N / 2) :
            p = p*-1 #if halfway shuffled list, the rest loses money        
        
        #personal profit is based on how many % of their wealth is invested #maybe kijken of normal dist beter werkt hier?
        #personal_p = p * (random.random()) #if uniform distribution is desired
        personal_p = p * func_bound_exp_p(0.4)  #if exponential with mean x is desired
        
        #appending the new balance to the list
        #people[Pidx].append(round(people[Pidx][-1] + personal_p*people[Pidx][-1],2)) #add new rounded entry to the persons money record
        people[Pidx].append(people[Pidx][-1] + personal_p*people[Pidx][-1]) #add new entry to the persons money record
    
    if distr > 0:
        people = func_redistr_wealth(people, distr)

    return people #return the list of people with new balances

def func_investments(r, distr = 0):
    '''
    runs r rounds of investments and plots them
    '''
    
    people = [[start_cash] for i in range(N)] #list of participants with their money history in a list (start with just 100)

    for i in range(r):
        people = func_inv_round(people, distr=distr)
    
    fig = go.Figure()
    for i in range(N):
        fig.add_trace(go.Scatter(x = range(r), y = people[i]))

    fig.show()  
    
    return people


    
people_0 = func_investments(rounds, distr=0)

In [ ]:
def func_sorted_time(people, t):
    '''
    takes a time as input and returns the sorted list of wealth from htl
    '''
    money_at_t = sorted([people[i][t] for i in range(N)])[::-1]
    return money_at_t

def func_powerlaw_t(people, t, log=False):
    '''
    creates a (lin-lin or log-log) plot for supplied t
    '''
    money_at_t = func_sorted_time(people, t)
    fpt_plot = go.Figure()
    fpt_plot.add_trace(go.Scatter(x = range(N), y = money_at_t))
    if log:
        fpt_plot.update_layout(xaxis_type="log", yaxis_type="log")
    fpt_plot.show()

    
    
# plot the lin-lin and log-log plot for the last investment round
func_powerlaw_t(people_0, -1) 
func_powerlaw_t(people_0, -1, log=True)

In [ ]:
def func_polyfit_t(people, t):
    '''
    creates a polyfit for t and returns the [a,b] and [error]
    '''
    data = func_sorted_time(people, t)
    x = np.array(range(len(data))) + 1 # Array with same size as data, starting from 1 to avoid pesky divide-by-zero errors
    polynomial_coefficients, residuals = np.polyfit(np.log(x),np.log(data), 1,full=True)[:2] # Returns coefficients of a polynomial of degree 1 (just a linear relation) with least square fit to data
    
    return polynomial_coefficients, residuals



#calculate the polyfit for the last investment round 
polycoeffs, error = func_polyfit_t(people_0, -1) 
print "polynomial coefficients [a,b] : {}\n               residual/error : {}".format(polycoeffs, error)

In [ ]:
'''turn the previous polynomial coeffs into a nice latex formula using ipynb markdown and python insertion'''

md("the best fitting power law for the log-log plot of the division of wealth is at the last t is:  \n  \n$$y = e^{} + x^{}$$".format(("{"+str(round(polycoeffs[1],3))+"}"), ("{"+str(round(polycoeffs[0],3))+"}")) )

In [ ]:
def func_plots_with_fit(people, t):
    '''
    plot the data of round t with the best fitting power law
    '''
    data = func_sorted_time(people, t)
    
    polynomial = np.poly1d(polycoeffs) #using poly1d to make a function from the variables gained previously
    
    x = np.array(range(len(data))) + 1 # Array with same size as data, starting from 1 to avoid pesky divide-by-zero errors

    
    log_plot = go.Figure()
    log_plot.add_trace(go.Scatter(x = range(N), y = data))
    log_plot.add_trace(go.Scatter(x = range(N), y = np.exp(polynomial(np.log(x)))))
    log_plot.update_layout(xaxis_type="log", yaxis_type="log")
    log_plot.show()

    lin_plot = go.Figure()
    lin_plot.add_trace(go.Scatter(x = range(N), y = data))
    lin_plot.add_trace(go.Scatter(x = range(N), y = np.exp(polynomial(np.log(x)))))
    lin_plot.show()
    
 

# plot the lin-lin and log-log plot for the last investment round
func_plots_with_fit(people_0, -1)

In [ ]:
def func_plot_a_vs_t(people):
    '''
    plots the a coefficient of each t's polyfit against t to visualize how powerlaw-y the data is. a=2 consitutes a powerlaw
    '''
    a_over_time = []

    for r in range(rounds):
        polycoeffs, error = func_polyfit_t(people, r)
        a_over_time.append(-1 * polycoeffs[0])

    alpha_plot = go.Figure()
    alpha_plot.add_trace(go.Scatter(x = range(r), y = a_over_time))

    alpha_plot.show()
    
    
    
func_plot_a_vs_t(people_0)

In [ ]:
def func_plot_e_vs_t(people):
    '''
    plots the residuals of each t's polyfit against t to visualize the acuracy of our fits
    '''
    e_over_time = []

    for r in range(rounds):
        polycoeffs, error = func_polyfit_t(people, r)
        e_over_time.append(error[0])

    e_plot = go.Figure()
    e_plot.add_trace(go.Scatter(x = range(r), y = e_over_time))

    e_plot.show()
    
    
    
func_plot_e_vs_t(people_0)

In [ ]:
def func_halfwaycash(people, t, info = False):
    '''
    given a t it returns the nth person so that everyone including them own closest to half of the wealth at that t
    using info prints the person in text with a bit of explanation
    '''
    ls_in = func_sorted_time(people, t)
    half_total_cash = sum(ls_in)/2 #what is halfway
    
    track_cash = 0
    wealthy = 0
    for idx, cash in enumerate(ls_in):
        #if the new itteration is closer to half than the previous
        if (abs(half_total_cash - track_cash) >
            abs(half_total_cash - (track_cash+cash))): 
            track_cash += cash
        else:
            if info:
                print("\n\nthere are {} rich that closest split the graph 50/50 left and right with €{} combined".format(idx, round(track_cash,2)))
                print("the nr. {} most wealthy person has €{}".format(idx, round(ls_in[idx-1],2)))
            return (round(idx/N *100,2))
    return

def func_plot_5050_vs_t(people):
    '''
    plots the amount least amount of people owning closest to half the wealth of each t against t to visualize the acuracy of our fits
    '''
    p50_over_time = []

    for r in range(rounds):
        p50_over_time.append(func_halfwaycash(people, r))

    p50_plot = go.Figure()
    p50_plot.add_trace(go.Scatter(x = range(r), y = p50_over_time))

    p50_plot.show()




#halfwaycash(func_powerlaw_t(10), info = True)
func_plot_5050_vs_t(people_0)   

In [ ]:
people_1 = func_investments(rounds, distr = 0.02)
func_plots_with_fit(people_1, -1)
#func_plot_a_vs_t(people_1)
#func_plot_e_vs_t(people_1)
#func_plot_5050_vs_t(people_1)